# RNN

## 经典RNN结构

| 网络结构 |        结构图示         | 应用场景举例                                                 |
| -------- | :---------------------: | ------------------------------------------------------------ |
| 1 vs N   | ![](./images/rnn-1ton.jpg)  | 1、从图像生成文字，输入为图像的特征，输出为一段句子<br />2、根据图像生成语音或音乐，输入为图像特征，输出为一段语音或音乐 |
| N vs 1   | ![](./images/rnn-nto1.jpg)  | 1、输出一段文字，判断其所属类别<br />2、输入一个句子，判断其情感倾向<br />3、输入一段视频，判断其所属类别 |
| N vs M   | ![](./images/rnn-ntom.jpg) | 1、机器翻译，输入一种语言文本序列，输出另外一种语言的文本序列<br />2、文本摘要，输入文本序列，输出这段文本序列摘要<br />3、阅读理解，输入文章，输出问题答案<br />4、语音识别，输入语音序列信息，输出文字序列 |

## RNN特点
1. RNNs主要用于处理序列数据
2. RNNs中当前单元的输出与之前步骤输出也有关，因此称之为循环神经网络
3. 标准的RNNs结构图，图中每个箭头代表做一次变换，也就是说箭头连接带有权值
4. 在标准的RNN结构中，隐层的神经元之间也是带有权值的，且权值共享。
5. 理论上，RNNs能够对任何长度序列数据进行处理。但是在实践中，为了降低复杂度往往假设当前的状态只与之前某几个时刻状态相关

## RNN公式
![](./images/rnnbp.png)

1. **前向传播**
  - 输入层到隐层
    - $h^{(t)}=\phi(Ux^{(t)}+Wh^{(t-1)}+b)$
    - $\phi()$为激活函数，一般会选择tanh函数，$b$为偏置
  - 隐层到输出层
    - $o^{(t)}=Vh^{(t)}+c$
  - 模型的预测输出
    - $\widehat{y}^{(t)}=\sigma(o^{(t)})$
    - $\sigma$为激活函数，通常RNN用于分类，故这里一般用softmax函数
  - 损失函数
    - $L^{(t)}=-\sum_{i}y_i^{(t)}\log(\widehat{y}_i^{(t)})$
    - $L=\sum_{t=1}^{n}L^{(t)}$

2. **反向传播**  BPTT（back-propagation through time）
  - 输出层到隐层
    - $\frac{\partial L^{(t)}}{\partial V}=\frac{\partial L^{(t)}}{\partial o^{(t)}}\cdot \frac{\partial o^{(t)}}{\partial V}$
    - $\frac{\partial L}{\partial V}=\sum_{t=1}^{n}\frac{\partial L^{(t)}}{\partial o^{(t)}}\cdot \frac{\partial o^{(t)}}{\partial V}$
  - 隐层到隐层
    - $\frac{\partial L}{\partial W}=\sum_{k=0}^{t}\frac{\partial L^{(t)}}{\partial o^{(t)}}\frac{\partial o^{(t)}}{\partial h^{(t)}}(\prod_{j=k+1}^{t}\frac{\partial h^{(j)}}{\partial h^{(j-1)}})\frac{\partial h^{(k)}}{\partial W}$
    - $\frac{\partial L}{\partial U}=\sum_{k=0}^{t}\frac{\partial L^{(t)}}{\partial o^{(t)}}\frac{\partial o^{(t)}}{\partial h^{(t)}}(\prod_{j=k+1}^{t}\frac{\partial h^{(j)}}{\partial h^{(j-1)}})\frac{\partial h^{(k)}}{\partial U}$


# LSTM
重复模块的区别
｜ 网络结构 |        结构图示         |
| -------- | :---------------------: | 
｜ RNN | ![](./images/rnn.png)  |
｜ LSTM | ![](./images/lstm.png)  |

### LSTM公式

| 网络结构 |        结构图示         | 公式 |
| -------- | :---------------------: | ---- |
| 遗忘门 | ![](./images/lstm-f.png)  | $$f^{(t)}=\sigma(W_{f}h^{(t-1)}+U_{f}x^{(t)}+b_{f})$$ |
| 输入门 | ![](./images/lstm-i.png)  | $$i^{(t)}=\sigma(W_{i}h^{(t-1)}+U_{i}x^{(t)}+b_{i})$$ $$a^{(t)}=\tanh(W_{a}h^{(t-1)}+U_{a}x^{(t)}+b_{a})$$ |
| 细胞状态更新 | ![](./images/lstm-c.png)  | $$c^{(t)}=f^{(t)}\odot c^{(t-1)}+i^{(t)}\odot {a}^{(t)}$$ |
| 输出门 | ![](./images/lstm-o.png)  | $$o^{(t)}=\sigma(W_{o}h^{(t-1)}+U_{o}x^{(t)}+b_{o})$$ $$h^{(t)}=o^{(t)}\odot \tanh(c^{(t)})$$ |

## Pytorch LSTM
![](./images/lstm-process.jpg)


### 网络结构

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable

lstm_seq = nn.LSTM(50, 100, num_layers=1) # 输入维度50，输出维度100
print(lstm_seq.weight_hh_l0.size())
print(lstm_seq.weight_ih_l0.size())
print(lstm_seq.bias_hh_l0.size())
print(lstm_seq.bias_ih_l0.size())

torch.Size([400, 100])
torch.Size([400, 50])
torch.Size([400])
torch.Size([400])


每组参数由4个部分组成，分别是遗忘门、输入门、细胞状态更新、输出门，每个部分的参数形状是`(hidden_size*4, input_size)`：
- `weight_hh_l0`参数代表第0层的隐藏状态到隐藏状态的权重，在公式中分别表示为$W_{f}$、$W_{i}$、$W_{a}$、$W_{o}$，隐藏层的大小是100，所以其形状是`(4*100, 100)`
- `weight_ih_l0`参数代表第0层的输入到隐藏状态的权重，在公式中分别表示为$U_{f}$、$U_{i}$、$U_{a}$、$U_{o}$，输入的大小是50，所以其形状是`(4*100, 50)`
- `bias_hh_l0`参数代表第0层的隐藏状态到隐藏状态的偏置
- `bias_ih_l0`参数代表第0层的输入到隐藏状态的偏置


In [9]:
class MyLSTM(nn.Module):
	def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
		super(MyLSTM, self).__init__()
		self.rnn = nn.LSTM(input_size, hidden_size, num_layers) # rnn
		self.reg = nn.Linear(hidden_size, output_size) # 回归

	def forward(self, x):
		x, _ = self.rnn(x) # (seq, batch, hidden)
		s, b, h = x.shape
		x = x.view(s*b, h) # 转换成线性层的输入格式
		x = self.reg(x)
		x = x.view(s, b, -1)
		return x

### 训练

In [ ]:
net = MyLSTM(2, 4)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)

epoch = 100
for t in range(epoch):
    # 前向传播
    out = net(Variable(train_x))
    loss = criterion(out, Variable(train_y))
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (t + 1) % 100 == 0: # 每 100 次输出结果
        print('Epoch: {}, Loss: {:.5f}'.format(t + 1, loss.data[0]))